In [1]:
import tensorflow as tf
from energyflow.archs import PFN
import training_functions
from training_functions import *
from sklearn.preprocessing import StandardScaler
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np

## Train, Val, Test Data

In [2]:
h5_filename = "split_test.hdf5"
h5_file = h5.File(h5_filename,'r')

In [3]:
label = "model_output_Oct3"  #Replace with your own variation!      
path = "./"+label

In [4]:
X_train = tf.data.Dataset.from_generator(
    generator(h5_filename,'train_hcal'),tf.float64)

X_val = tf.data.Dataset.from_generator(
    generator(h5_filename,'val_hcal'),tf.float64)

X_test = tf.data.Dataset.from_generator(
    generator(h5_filename,'test_hcal'),tf.float64)

Y_train = tf.data.Dataset.from_generator(
    generator(h5_filename,'train_mc'),tf.float64)

Y_val = tf.data.Dataset.from_generator(
    generator(h5_filename,'val_mc'),tf.float64)

Y_test = tf.data.Dataset.from_generator(
    generator(h5_filename,'test_mc'),tf.float64)

2022-10-05 15:29:54.525997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/global/home/users/ftoralesacosta/user_pkg/usr/lib:/global/home/users/ftoralesacosta/user_pkg/usr/lib64:/lib:/lib64:/usr/lib:/usr/lib64
2022-10-05 15:29:54.526024: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-05 15:29:54.526762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions

In [5]:
#tf_dataset = Y_train
#scalar_from_generator(tf_dataset,10,100)                                                              

## Hyper Parameters

In [6]:
input_dim = h5_file['train_hcal'].shape[-1]
learning_rate = 1e-4
dropout_rate = 0.1
batch_size = 1000
N_Epochs = 400
patience = 20
N_Latent = 128
shuffle_split = True #Turn FALSE for images!
train_shuffle = True #Turn TRUE for images!
Y_scalar = True
loss = 'mse' #'mae' #'swish'

## Define Model + CallBacks

In [7]:
Phi_sizes, F_sizes = (100, 100, N_Latent), (100, 100, 100)
output_act, output_dim = 'linear', 1 #Train to predict error

pfn = PFN(input_dim=input_dim, 
          Phi_sizes=Phi_sizes, 
          F_sizes=F_sizes, 
          output_act=output_act, 
          output_dim=output_dim, 
          loss=loss, 
          latent_dropout=dropout_rate,
          F_dropouts=dropout_rate,
          optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, 1861)  0           []                               
                                ]                                                                 
                                                                                                  
 tdist_0 (TimeDistributed)      (None, None, 100)    186200      ['input[0][0]']                  
                                                                                                  
 activation (Activation)        (None, None, 100)    0           ['tdist_0[0][0]']                
                                                                                                  
 tdist_1 (TimeDistributed)      (None, None, 100)    10100       ['activation[0][0]']         

In [8]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_decay,verbose=0)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience)

Tensorflow needs a generator that returns a tuple (train, target), so I'm trying the below

In [9]:
training_generator = tf.data.Dataset.from_generator(
    train_target_generator(h5_filename,'train_hcal','train_mc'),
    output_shapes=(tf.TensorShape([None,None]),[None]),
    output_types=(tf.float64, tf.float64))

val_generator = tf.data.Dataset.from_generator(
    train_target_generator(h5_filename,'val_hcal','val_mc'),
    output_shapes=(tf.TensorShape([None,None]),[None]),
    output_types=(tf.float64, tf.float64))

training_generator.batch(batch_size)
val_generator.batch(batch_size)

<BatchDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float64, name=None), TensorSpec(shape=(None, None), dtype=tf.float64, name=None))>

In [10]:
for data in training_generator.batch(batch_size):
    print(data)
    break

(<tf.Tensor: shape=(1000, 4, 1861), dtype=float64, numpy=
array([[[-0.20597402, -0.25450045,  0.12657475, ...,         nan,
                 nan,         nan],
        [ 0.40956907,  0.34803294,  0.32550916, ...,         nan,
                 nan,         nan],
        [-0.51216165, -0.61735187, -0.53439095, ...,         nan,
                 nan,         nan],
        [-1.49957625, -1.73549547, -1.73549547, ...,         nan,
                 nan,         nan]],

       [[-0.34774731, -0.3572623 , -0.34965031, ...,         nan,
                 nan,         nan],
        [ 1.43477641,  1.43477641,  1.43477641, ...,         nan,
                 nan,         nan],
        [ 0.02570211,  0.02570211,  0.02570211, ...,         nan,
                 nan,         nan],
        [-1.73549547, -1.65685573, -1.57821599, ...,         nan,
                 nan,         nan]],

       [[-0.34298982, -0.33632933, -0.35298055, ...,         nan,
                 nan,         nan],
        [-1.48641344

# FIT

In [11]:
the_fit = pfn.fit(
    training_generator,
    epochs=N_Epochs,
    batch_size=batch_size,    
    callbacks=[lr_scheduler, early_stopping],
    validation_data=val_generator,
    verbose=1
)

Epoch 1/400


ValueError: in user code:

    File "/global/home/users/ftoralesacosta/anaconda3/envs/tin/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/global/home/users/ftoralesacosta/anaconda3/envs/tin/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/global/home/users/ftoralesacosta/anaconda3/envs/tin/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/global/home/users/ftoralesacosta/anaconda3/envs/tin/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/global/home/users/ftoralesacosta/anaconda3/envs/tin/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtere

In [ ]:
pfn.layers
pfn.save("%s/energy_regression.h5"%(path))
mypreds = pfn.predict(X_test,batch_size=400)

if (Y_scalar):
    mypreds = mypreds*Y_StDev + Y_Mean
    Y_test  =  Y_test*Y_StDev + Y_Mean
    
np.save("%s/predictions.npy"%(path),mypreds)
np.save("%s/y_test.npy"%(path),Y_test)
np.save("%s/x_test.npy"%(path),X_test)

In [ ]:
tf_dataset = X_train

scaler = StandardScaler()

for data,ibatch in zip(tf_dataset.batch(1000),range(0,100)):
    scaler.partial_fit(data.numpy().transpose(0,2,1).reshape(-1,4))
    print("mean = ",scaler.mean_,"+/-",np.sqrt(scaler.var_))

In [ ]:
# Make a dict for train,test,val mean and stdev. Also for target/labels. Remember, dataset needs to return tuple of (input, target)